In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback
data = pd.read_csv('/kaggle/input/binary-class/binary_class.csv')
model_name = "cointegrated/rubert-tiny2"
device = 'cuda' if torch.cuda.is_available else 'cpu'

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
X = list(data["Текст сообщения"])
y = list(data["Разметка"])

In [11]:
# X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=128)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=128)

In [20]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [21]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average = 'macro')
    precision = precision_score(y_true=labels, y_pred=pred, average = 'macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average = 'macro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer
args = TrainingArguments(
    output_dir="/output",
    evaluation_strategy="steps",
    eval_steps=50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=128,
    num_train_epochs=30,
    weight_decay=0.05,
    learning_rate = 1e-5,
    warmup_ratio=0.2,
    lr_scheduler_type = 'cosine',
    seed=20222022,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,    
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,No log,0.691619,0.538653,0.518103,0.516819,0.514181
100,No log,0.687403,0.600998,0.588164,0.559166,0.539054
150,No log,0.682973,0.603491,0.606708,0.550839,0.508885
200,No log,0.673474,0.613466,0.749213,0.548284,0.467400
250,No log,0.664363,0.635910,0.758370,0.575350,0.516947
300,No log,0.651356,0.648379,0.769292,0.589969,0.540933
350,No log,0.636103,0.655860,0.740504,0.601742,0.565336
400,No log,0.618500,0.680798,0.693023,0.646733,0.642662
450,No log,0.602078,0.710723,0.712282,0.687821,0.690290
500,0.655400,0.582547,0.705736,0.705654,0.683473,0.685803


TrainOutput(global_step=3030, training_loss=0.4042874342537556, metrics={'train_runtime': 80.4464, 'train_samples_per_second': 597.416, 'train_steps_per_second': 37.665, 'total_flos': 88601096816640.0, 'train_loss': 0.4042874342537556, 'epoch': 30.0})

In [23]:
# model.save_pretrained('bin(importance_classification)')

In [42]:
X_test = tokenizer(['угабуга'], padding=True, truncation=True, max_length=128)

In [43]:
test_dataset = Dataset(X_test)

In [44]:
test_trainer = Trainer(model)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [45]:
raw_pred, _, _ = test_trainer.predict(test_dataset) # Классификация запроса
y_pred = torch.argmax(torch.softmax(torch.tensor(raw_pred), dim=-1), axis=1)

In [47]:
model.save_pretrained('bin_model(importance_classification)')
tokenizer.save_pretrained('bin_tokenizer(importance_classification)')

('bin_tokenizer(importance_classification)/tokenizer_config.json',
 'bin_tokenizer(importance_classification)/special_tokens_map.json',
 'bin_tokenizer(importance_classification)/vocab.txt',
 'bin_tokenizer(importance_classification)/added_tokens.json')

In [49]:
import os
import zipfile
tokenizer_path = '/kaggle/working/bin_tokenizer(importance_classification)'
model_path = '/kaggle/working/bin_model(importance_classification)'

archive_name = "bert_model_archive.zip"
with zipfile.ZipFile(archive_name, "w") as archive:
    # Add model files
    archive.write(os.path.join(model_path, "config.json"), "config.json")
    archive.write(os.path.join(model_path, "model.safetensors"), "model.safetensors")
    # Add tokenizer files
    archive.write(os.path.join(tokenizer_path, "vocab.txt"), "vocab.txt")
    archive.write(os.path.join(tokenizer_path, "special_tokens_map.json"), "special_tokens_map.json")
    archive.write(os.path.join(tokenizer_path, "tokenizer_config.json"), "tokenizer_config.json")